In [25]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

import os


In [26]:

# Especificar la ruta a la carpeta que contiene los archivos CSV
ruta = os.path.abspath('../data/modelos_entrenamiento/')

# Cargar los datos desde los archivos CSV en DataFrames
X_train = pd.read_csv(os.path.join(ruta, 'X_train.csv'))
y_train = pd.read_csv(os.path.join(ruta, 'y_train.csv'))
X_val = pd.read_csv(os.path.join(ruta, 'X_val.csv'))
y_val = pd.read_csv(os.path.join(ruta, 'y_val.csv'))
X_test = pd.read_csv(os.path.join(ruta, 'X_test.csv'))
y_test = pd.read_csv(os.path.join(ruta, 'y_test.csv'))

# Verificar que las dimensiones de X e y coincidan para cada conjunto de datos
assert X_train.shape[0] == y_train.shape[0], "Error: El número de filas en X_train y y_train no coincide."
assert X_val.shape[0] == y_val.shape[0], "Error: El número de filas en X_val y y_val no coincide."
assert X_test.shape[0] == y_test.shape[0], "Error: El número de filas en X_test y y_test no coincide."

# Verificar que las columnas tengan nombres correctos y consistentes
assert 'satisfaction' in y_train.columns, "Error: El archivo y_train.csv debe tener una columna llamada 'satisfaction'."
assert 'satisfaction' in y_val.columns, "Error: El archivo y_val.csv debe tener una columna llamada 'satisfaction'."
assert 'satisfaction' in y_test.columns, "Error: El archivo y_test.csv debe tener una columna llamada 'satisfaction'."

print("Todos los archivos tienen un número de filas coincidente entre características y etiquetas, y la columna objetivo es 'satisfaction'.")



Todos los archivos tienen un número de filas coincidente entre características y etiquetas, y la columna objetivo es 'satisfaction'.


In [27]:
# Separar las características (X) de la variable objetivo (y)
# Aquí asumimos que X_train, X_val, y X_test ya son DataFrames que contienen solo las características
# Si estos DataFrames incluyen la columna 'satisfaction', habría que eliminarla.

# Separar la variable objetivo (y)
y_train = y_train['satisfaction']
y_val = y_val['satisfaction']
y_test = y_test['satisfaction']

print("Características y etiquetas separadas correctamente.")


Características y etiquetas separadas correctamente.


In [28]:
# Verificar tipos de datos de las etiquetas
print("Tipo de datos de y_train:", y_train.dtypes)
print("Tipo de datos de y_val:", y_val.dtypes)
print("Tipo de datos de y_test:", y_test.dtypes)

# Convertir las etiquetas a números (si es necesario)
# Este paso solo se ejecutará si las etiquetas no son numéricas
y_train = pd.to_numeric(y_train, errors='coerce')
y_val = pd.to_numeric(y_val, errors='coerce')
y_test = pd.to_numeric(y_test, errors='coerce')

# Confirmar la conversión
print("Tipos de datos después de la conversión:")
print("Tipo de datos de y_train:", y_train.dtypes)
print("Tipo de datos de y_val:", y_val.dtypes)
print("Tipo de datos de y_test:", y_test.dtypes)


Tipo de datos de y_train: int64
Tipo de datos de y_val: int64
Tipo de datos de y_test: int64
Tipos de datos después de la conversión:
Tipo de datos de y_train: int64
Tipo de datos de y_val: int64
Tipo de datos de y_test: int64


In [29]:
from catboost import CatBoostClassifier
from sklearn.metrics import recall_score, f1_score, roc_auc_score

# Inicializar el modelo CatBoostClassifier
model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, verbose=100)

# Entrenar el modelo utilizando el conjunto de entrenamiento y validación
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

# Evaluar el modelo en el conjunto de prueba (test)
accuracy = model.score(X_test, y_test)
print(f"Exactitud del modelo en el conjunto de prueba: {accuracy:.2f}")



0:	learn: 0.5642258	test: 0.5643419	best: 0.5643419 (0)	total: 12ms	remaining: 5.99s
100:	learn: 0.0954957	test: 0.1020326	best: 0.1020326 (100)	total: 1.4s	remaining: 5.52s
200:	learn: 0.0812890	test: 0.0933198	best: 0.0933198 (200)	total: 3.06s	remaining: 4.56s
300:	learn: 0.0732948	test: 0.0903986	best: 0.0903844 (299)	total: 4.28s	remaining: 2.83s
400:	learn: 0.0675110	test: 0.0891242	best: 0.0891138 (378)	total: 5.46s	remaining: 1.35s
499:	learn: 0.0624786	test: 0.0884543	best: 0.0884543 (499)	total: 6.64s	remaining: 0us

bestTest = 0.08845431341
bestIteration = 499

Exactitud del modelo en el conjunto de prueba: 0.96


In [30]:
# Realizar predicciones en los conjuntos de entrenamiento y prueba
catboost_train_preds = model.predict(X_train)
catboost_test_preds = model.predict(X_test)

# Calcular las probabilidades predichas para el conjunto de entrenamiento y prueba
catboost_train_proba = model.predict_proba(X_train)[:, 1]
catboost_test_proba = model.predict_proba(X_test)[:, 1]

# Calcular las métricas de evaluación para los conjuntos de entrenamiento y prueba
catboost_train_recall = recall_score(y_train, catboost_train_preds)
catboost_test_recall = recall_score(y_test, catboost_test_preds)

catboost_train_f1 = f1_score(y_train, catboost_train_preds)
catboost_test_f1 = f1_score(y_test, catboost_test_preds)

catboost_train_auc = roc_auc_score(y_train, catboost_train_proba)
catboost_test_auc = roc_auc_score(y_test, catboost_test_proba)

# Imprimir las métricas de evaluación para analizar el rendimiento del modelo
print(f"CatBoost - Recall en Entrenamiento: {catboost_train_recall:.2f}")
print(f"CatBoost - Recall en Prueba: {catboost_test_recall:.2f}")
print(f"CatBoost - F1 Score en Entrenamiento: {catboost_train_f1:.2f}")
print(f"CatBoost - F1 Score en Prueba: {catboost_test_f1:.2f}")
print(f"CatBoost - AUC en Entrenamiento: {catboost_train_auc:.2f}")
print(f"CatBoost - AUC en Prueba: {catboost_test_auc:.2f}")


CatBoost - Recall en Entrenamiento: 0.96
CatBoost - Recall en Prueba: 0.94
CatBoost - F1 Score en Entrenamiento: 0.97
CatBoost - F1 Score en Prueba: 0.96
CatBoost - AUC en Entrenamiento: 1.00
CatBoost - AUC en Prueba: 1.00


In [32]:
# Importar las funciones necesarias desde sklearn.metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Realizar predicciones en los conjuntos de entrenamiento, validación y prueba
catboost_train_preds = model.predict(X_train)
catboost_val_preds = model.predict(X_val)
catboost_test_preds = model.predict(X_test)

# Calcular las probabilidades predichas para los conjuntos de entrenamiento, validación y prueba
catboost_train_proba = model.predict_proba(X_train)[:, 1]
catboost_val_proba = model.predict_proba(X_val)[:, 1]
catboost_test_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas para el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, catboost_train_preds)
train_precision = precision_score(y_train, catboost_train_preds, pos_label=1)
train_recall = recall_score(y_train, catboost_train_preds, pos_label=1)
train_f1 = f1_score(y_train, catboost_train_preds, pos_label=1)
train_auc = roc_auc_score(y_train, catboost_train_proba)
train_report = classification_report(y_train, catboost_train_preds)

# Calcular métricas para el conjunto de validación
val_accuracy = accuracy_score(y_val, catboost_val_preds)
val_precision = precision_score(y_val, catboost_val_preds, pos_label=1)
val_recall = recall_score(y_val, catboost_val_preds, pos_label=1)
val_f1 = f1_score(y_val, catboost_val_preds, pos_label=1)
val_auc = roc_auc_score(y_val, catboost_val_proba)
val_report = classification_report(y_val, catboost_val_preds)

# Calcular métricas para el conjunto de prueba
test_accuracy = accuracy_score(y_test, catboost_test_preds)
test_precision = precision_score(y_test, catboost_test_preds, pos_label=1)
test_recall = recall_score(y_test, catboost_test_preds, pos_label=1)
test_f1 = f1_score(y_test, catboost_test_preds, pos_label=1)
test_auc = roc_auc_score(y_test, catboost_test_proba)
test_report = classification_report(y_test, catboost_test_preds)

# Imprimir métricas para el conjunto de entrenamiento
print(f"--- Métricas para el Conjunto de Entrenamiento ---")
print(f"Exactitud en Entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en Entrenamiento: {train_precision:.2f}")
print(f"Recall en Entrenamiento: {train_recall:.2f}")
print(f"F1 Score en Entrenamiento: {train_f1:.2f}")
print(f"AUC en Entrenamiento: {train_auc:.2f}")
print("Reporte de Clasificación en Entrenamiento:\n", train_report)

# Imprimir métricas para el conjunto de validación
print(f"--- Métricas para el Conjunto de Validación ---")
print(f"Exactitud en Validación: {val_accuracy:.2f}")
print(f"Precisión en Validación: {val_precision:.2f}")
print(f"Recall en Validación: {val_recall:.2f}")
print(f"F1 Score en Validación: {val_f1:.2f}")
print(f"AUC en Validación: {val_auc:.2f}")
print("Reporte de Clasificación en Validación:\n", val_report)

# Imprimir métricas para el conjunto de prueba
print(f"--- Métricas para el Conjunto de Prueba ---")
print(f"Exactitud en Prueba: {test_accuracy:.2f}")
print(f"Precisión en Prueba: {test_precision:.2f}")
print(f"Recall en Prueba: {test_recall:.2f}")
print(f"F1 Score en Prueba: {test_f1:.2f}")
print(f"AUC en Prueba: {test_auc:.2f}")
print("Reporte de Clasificación en Prueba:\n", test_report)


--- Métricas para el Conjunto de Entrenamiento ---
Exactitud en Entrenamiento: 0.98
Precisión en Entrenamiento: 0.98
Recall en Entrenamiento: 0.96
F1 Score en Entrenamiento: 0.97
AUC en Entrenamiento: 1.00
Reporte de Clasificación en Entrenamiento:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98     35261
           1       0.98      0.96      0.97     26895

    accuracy                           0.98     62156
   macro avg       0.98      0.97      0.98     62156
weighted avg       0.98      0.98      0.98     62156

--- Métricas para el Conjunto de Validación ---
Exactitud en Validación: 0.96
Precisión en Validación: 0.97
Recall en Validación: 0.94
F1 Score en Validación: 0.96
AUC en Validación: 0.99
Reporte de Clasificación en Validación:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97     11781
           1       0.97      0.94      0.96      8938

    accuracy                  